# Search Similar 3D Meshes

In this tutorial, we will learn how to build a 3D mesh search pipeline with Jina. In particular, we will be building a search pipeline for 3D models in GLB format.

Just like other data types, the 3D meshes search pipeline consists of **loading**, **encoding** and **indexing** the data. We can search the data after they are indexed.

## Prerequisites

Let's first install the following PyPI dependencies:

In [1]:
!pip install tensorflow trimesh pyrender

     |████████████████████████████████| 641 kB 2.1 MB/s 
     |████████████████████████████████| 1.2 MB 10.2 MB/s 
     |████████████████████████████████| 462 kB 34.5 MB/s 
     |████████████████████████████████| 2.2 MB 33.0 MB/s 
     |████████████████████████████████| 890 kB 38.0 MB/s 
  Created wheel for PyOpenGL: filename=PyOpenGL-3.1.0-py3-none-any.whl size=1745210 sha256=5fc4054ec869fe80d643a47159321433739553fadca9f8f7ed64c881d571fb87
  Stored in directory: /root/.cache/pip/wheels/c6/83/cb/af51a0c06c33d08537b941bbfc87469e8a3c68d05f77a6a212
Successfully built PyOpenGL
  Attempting uninstall: PyOpenGL
    Found existing installation: PyOpenGL 3.1.6
    Uninstalling PyOpenGL-3.1.6:
      Successfully uninstalled PyOpenGL-3.1.6


In [3]:
!pip install docarray

     |████████████████████████████████| 611 kB 1.8 MB/s 
     |████████████████████████████████| 224 kB 38.5 MB/s 
     |████████████████████████████████| 51 kB 4.1 MB/s 
  Created wheel for docarray: filename=docarray-0.9.16-py3-none-any.whl size=650532 sha256=8a41017823a07cae2e0cb1e7e3e290afae7e01d40d4d8999888b5b9d1a222def
  Stored in directory: /root/.cache/pip/wheels/66/c7/44/e3b1cd32a45cc9185e929519dc1c168ea4e910bbcfb360f62f
Successfully built docarray


In [4]:
!pip install jina

     |████████████████████████████████| 280 kB 2.1 MB/s 
     |████████████████████████████████| 54 kB 2.7 MB/s 
     |████████████████████████████████| 3.8 MB 8.4 MB/s 
     |████████████████████████████████| 1.8 MB 32.1 MB/s 
     |████████████████████████████████| 146 kB 42.6 MB/s 
     |████████████████████████████████| 1.1 MB 37.0 MB/s 
     |████████████████████████████████| 3.6 MB 26.5 MB/s 
     |████████████████████████████████| 1.1 MB 46.0 MB/s 
     |████████████████████████████████| 10.9 MB 31.4 MB/s 
     |████████████████████████████████| 596 kB 35.5 MB/s 
     |████████████████████████████████| 53 kB 1.8 MB/s 
     |████████████████████████████████| 111 kB 46.0 MB/s 
     |████████████████████████████████| 144 kB 48.5 MB/s 
     |████████████████████████████████| 271 kB 40.2 MB/s 
     |████████████████████████████████| 94 kB 1.2 MB/s 
     |████████████████████████████████| 54 kB 2.5 MB/s 
     |████████████████████████████████| 58 kB 4.9 MB/s 
     |███████████████████

## Load GLB data

First, given a `glb` file, how do we load and craft the `glb` into a Document so that we can process and encode?
Let's use `trimesh` to build an Executor for this.

In [1]:
import trimesh
from typing import Optional
from docarray import Document, DocumentArray
from jina import Executor, requests

def as_mesh(scene: trimesh.Scene) -> Optional[trimesh.Trimesh]:
    if len(scene.geometry) == 0:
        return None
    return trimesh.util.concatenate(
        tuple(trimesh.Trimesh(vertices=g.vertices, faces=g.faces)
            for g in scene.geometry.values()))


class GlbCrafter(Executor):
    @requests(on=['/index', '/search'])
    def craft(self, docs: DocumentArray, **kwargs):
        for d in docs:
            mesh = trimesh.load_mesh(d.uri)
            d.tensor = as_mesh(trimesh.load_mesh(d.uri)).sample(2048)

We first load the data of each `glb` file as Python object.
We will use the `trimesh` package to represents the `glb` data in the form of triangular meshes.
The loaded object is of type `trimesh.Scene` which may contain one or more triangular mesh geometries.
We combine all the meshes in the `Scene` to create a single `Trimesh` using `as_mesh`.
Then we can sample surfaces from a single mesh geometry.
The sampled surface will be made from 2048 points in 3D space and hence the shape of the `ndarray` representing each 3D model will be `(2048, 3)`.

## Encode 3D Model

Once we convert each `glb` model into an `ndarray`, encoding the inputs becomes straightforward.
We will use our pre-trained `pointnet` to encode the data. The model looks like:

In [2]:
def get_model(ckpt_path):
    import numpy as np
    import tensorflow as tf
    from tensorflow import keras
    from keras import layers
    
    def conv_bn(x, filters):
        x = layers.Conv1D(filters, kernel_size=1, padding='valid')(x)
        x = layers.BatchNormalization(momentum=0.0)(x)
        return layers.Activation('relu')(x)
    
    
    def dense_bn(x, filters):
        x = layers.Dense(filters)(x)
        x = layers.BatchNormalization(momentum=0.0)(x)
        return layers.Activation('relu')(x)
    
    
    def tnet(inputs, num_features):
        class OrthogonalRegularizer(keras.regularizers.Regularizer):
            def __init__(self, num_features_, l2reg=0.001):
                self.num_features = num_features_
                self.l2reg = l2reg
                self.eye = tf.eye(self.num_features)
    
            def __call__(self, x):
                x = tf.reshape(x, (-1, self.num_features, self.num_features))
                xxt = tf.tensordot(x, x, axes=(2, 2))
                xxt = tf.reshape(xxt, (-1, self.num_features, self.num_features))
                return tf.reduce_sum(self.l2reg * tf.square(xxt - self.eye))
    
            def get_config(self):
                return {'num_features': self.num_features,
                        'l2reg': self.l2reg,
                        'eye': self.eye.numpy()}
    
        bias = keras.initializers.Constant(np.eye(num_features).flatten())
        reg = OrthogonalRegularizer(num_features)
    
        x = conv_bn(inputs, 32)
        x = conv_bn(x, 64)
        x = conv_bn(x, 512)
        x = layers.GlobalMaxPooling1D()(x)
        x = dense_bn(x, 256)
        x = dense_bn(x, 128)
        x = layers.Dense(
            num_features * num_features,
            kernel_initializer='zeros',
            bias_initializer=bias,
            activity_regularizer=reg,
        )(x)
        feat_T = layers.Reshape((num_features, num_features))(x)
        return layers.Dot(axes=(2, 1))([inputs, feat_T])

    inputs = keras.Input(shape=(2048, 3))
    x = tnet(inputs, 3)
    x = conv_bn(x, 32)
    x = conv_bn(x, 32)
    x = tnet(x, 32)
    x = conv_bn(x, 32)
    x = conv_bn(x, 64)
    x = layers.GlobalMaxPooling1D()(x)
    x = dense_bn(x, 128)
    x = layers.Dropout(0.3)(x)
    outputs = layers.Dense(1, activation='softmax')(x)
    model = keras.Model(inputs=inputs, outputs=outputs, name='pointnet')
    intermediate_layer_model = keras.Model(inputs=model.input,
                                           outputs=model.get_layer(f'dense_1').output)
    intermediate_layer_model.load_weights(ckpt_path)
    return intermediate_layer_model

With the above model, we can then build our `pointnet` executor:

In [3]:
class PNEncoder(Executor):
    def __init__(self, ckpt_path: str, **kwargs):
        super().__init__(**kwargs)
        self.embedding_model = get_model(ckpt_path=ckpt_path)

    @requests(on=['/index', '/search'])
    def encode(self, docs: DocumentArray, **kwargs):
        docs.embeddings = self.embedding_model.predict(docs.tensors)

<div class="alert alert-block alert-info">
<b>Tip:</b>  Instead of iterating over each doc to set its embedding, we can directly get the tensors of all docs in `docs` at once by using the attribute `blobs` and set the embeddings of all docs in `docs` at once by using the attribute `embeddings`.</div>

## Index the data

Let's also build an indexer to index the data.

In [4]:
class MyIndexer(Executor):
    _docs = DocumentArray()

    @requests(on='/index')
    def index(self, docs: DocumentArray, **kwargs):
        self._docs.extend(docs)

    @requests(on='/search')
    def search(self, docs: DocumentArray, **kwargs):
        docs.match(self._docs, limit=5)

The above indexer simply uses `DocumentArray` to store all the index docs and leverages the `match` function of `DocumentArray` to match the query with docs indexed.

## Visualize 3D Model

Finally, let's also build the `GlbVisualizer` to visualize the results.

In [8]:
import pyrender
import pyglet
from pyglet import clock
from pyglet.gl import Config
from pyrender import Viewer


def _init_and_start_app(self):
        TARGET_OPEN_GL_MAJOR = 4  # Target OpenGL Major Version
        TARGET_OPEN_GL_MINOR = 1
        MIN_OPEN_GL_MAJOR = 3     # Minimum OpenGL Major Version
        MIN_OPEN_GL_MINOR = 3     # Minimum OpenGL Minor Version
        confs = [Config(sample_buffers=1, samples=4,
                        depth_size=24,
                        double_buffer=True,
                        major_version=TARGET_OPEN_GL_MAJOR,
                        minor_version=TARGET_OPEN_GL_MINOR),
                 Config(depth_size=24,
                        double_buffer=True,
                        major_version=TARGET_OPEN_GL_MAJOR,
                        minor_version=TARGET_OPEN_GL_MINOR),
                 Config(sample_buffers=1, samples=4,
                        depth_size=24,
                        double_buffer=True,
                        major_version=MIN_OPEN_GL_MAJOR,
                        minor_version=MIN_OPEN_GL_MINOR),
                 Config(depth_size=24,
                        double_buffer=True,
                        major_version=MIN_OPEN_GL_MAJOR,
                        minor_version=MIN_OPEN_GL_MINOR)]
        for conf in confs:
            try:
                super(Viewer, self).__init__(config=conf, resizable=True,
                                             width=self._viewport_size[0],
                                             height=self._viewport_size[1])
                break
            except pyglet.window.NoSuchConfigException:
                pass

        if not self.context:
            raise ValueError('Unable to initialize an OpenGL 3+ context')

        clock.schedule_interval(
            Viewer._time_event, 1.0 / self.viewer_flags['refresh_rate'], self
        )
        self.switch_to()
        self.set_caption(self.viewer_flags['window_title'])


class GlbVisualizer:
    def __init__(self, search_doc, matches=None):
        self.search_doc = search_doc
        self.matches = matches
        self.orig_func = pyrender.Viewer._init_and_start_app
        pyrender.Viewer._init_and_start_app = _init_and_start_app

    def visualize(self):
        self.add(self.search_doc.uri, 'Query Doc')
        if self.matches:
            for i, match in enumerate(self.matches, start=1):
                self.add(match.uri, f'Top {i} Match')
        pyglet.app.run()

    def add(self, uri, title):
        fuze_trimesh = as_mesh(trimesh.load(uri))
        mesh = pyrender.Mesh.from_trimesh(fuze_trimesh)
        scene = pyrender.Scene()
        scene.add(mesh)

        pyrender.Viewer(
            scene,
            use_raymond_lighting=True,
            viewer_flags={
                'rotate': True,
                'window_title': title, 
                'caption': [{
                    'font_name': 'OpenSans-Regular',
                    'font_pt': 30,
                    'color': None,
                    'scale': 1.0,
                    'location': 4,
                    'text': title
                }]
            },
        )
        
    def __del__(self):
        pyrender.Viewer._init_and_start_app = self.orig_func

The visualizer uses `pyrender` to render the query and match results.
Since we want to display multiple models at once, we need to patch the `_init_and_start_app` function to delay the start of pyrender app after all viewers are initialized.


## Index, Search and Visualize Data

Download the pre-trained PNEncoder model [here](https://github.com/jina-ai/example-3D-model/tree/main/executors/pn_encoder/ckpt) into `model/ckpt`.
Also, store your index/search data in `data/`.
We can then put the executors into a flow and use the flow to perform indexing and searching.
Finally, we use the `GlbVisualizer` built earlier to visualize our data.


<div class="alert alert-block alert-info">
<b>Getting data:</b> If you dont have .glb data already, you can download some samples from here (https://shapenet.org/), after creating a free account.</div>

In [9]:
from jina import Flow

with Flow().add(uses=GlbCrafter).add(uses=PNEncoder, uses_with={'ckpt_path': 'model/ckpt/ckpt_True'}).add(uses=MyIndexer) as f:
    f.index(DocumentArray.from_files('data/*.glb'))
    results = f.search(Document(uri='data/ShapeNetV2_airplane_aeroplane_plane_0.glb'), return_results=True)
    doc = results[0]
    # visualize top 3 matches, since we also index query doc, exclude the top 1 match as it is the query doc
    visualizer = GlbVisualizer(doc, matches=doc.matches).visualize()

Output()

executor1/rep-0@523[C]:can not load the executor from PNEncoder
executor1/rep-0@523[E]:ExecutorFailToLoad() during <class 'jina.serve.runtimes.worker.WorkerRuntime'> initialization
 add "--quiet-error" to suppress the exception details
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/jina/serve/runtimes/request_handlers/data_request_handler.py", line 49, in _load_executor
    extra_search_paths=self.args.extra_search_paths,
  File "/usr/local/lib/python3.7/dist-packages/jina/jaml/__init__.py", line 726, in load_config
    return JAML.load(tag_yml, substitute=False)
  File "/usr/local/lib/python3.7/dist-packages/jina/jaml/__init__.py", line 166, in load
    r = yaml.load(stream, Loader=JinaLoader)
  File "/usr/local/lib/python3.7/dist-packages/yaml/__init__.py", line 81, in load
    return loader.get_single_data()
  File "/usr/local/lib/python3.7/dist-packages/yaml/constructor.py", line 51, in get_single_data
    return self.construct_document(node)
  Fi

╭──────────────────────────── Traceback (most recent call last) ────────────────────────────╮
│                                                                                           │
│ /usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882 in run_code  │
│                                                                                           │
│   2879 │   │   │   try:                                                                   │
│   2880 │   │   │   │   self.hooks.pre_run_code_hook()                                     │
│   2881 │   │   │   │   #rprint('Running code', repr(code_obj)) # dbg                      │
│ ❱ 2882 │   │   │   │   exec(code_obj, self.user_global_ns, self.user_ns)                  │
│   2883 │   │   │   finally:                                                               │
│   2884 │   │   │   │   # Reset our crash handler in place                                 │
│   2885 │   │   │   │   sys.excepthook = old_excepthook                                    │
│ <ipython-input-9-560c91677931>:3 in <module>                                              │
│                                                                                           │
│ /usr/local/lib/python3.7/dist-packages/jina/orchestrate/flow/base.py:1075 in __enter__    │
│                                                                                           │
│   1072 │                                                                                  │
│   1073 │   def __enter__(self):                                                           │
│   1074 │   │   with CatchAllCleanupContextManager(self):                                  │
│ ❱ 1075 │   │   │   return self.start()                                                    │
│   1076 │                                                                                  │
│   1077 │   def __exit__(self, exc_type, exc_val, exc_tb):                                 │
│   1078 │   │   if hasattr(self, '_stop_event'):                                           │
│                                                                                           │
│ /usr/local/lib/python3.7/dist-packages/jina/orchestrate/flow/base.py:1122 in start        │
│                                                                                           │
│   1119 │   │   │   if not v.external:                                                     │
│   1120 │   │   │   │   self.enter_context(v)                                              │
│   1121 │   │                                                                              │
│ ❱ 1122 │   │   self._wait_until_all_ready()                                               │
│   1123 │   │                                                                              │
│   1124 │   │   self._build_level = FlowBuildLevel.RUNNING                                 │
│   1125                                                                                    │
│                                                                                           │
│ /usr/local/lib/python3.7/dist-packages/jina/orchestrate/flow/base.py:1202 in              │
│ _wait_until_all_ready                                                                     │
│                                                                                           │
│   1199 │   │   │   │   │   f'Flow is aborted due to {error_deployments} can not be starte │
│   1200 │   │   │   │   )                                                                  │
│   1201 │   │   │   │   self.close()                                                       │
│ ❱ 1202 │   │   │   │   raise RuntimeFailToStart                                           │
│   1203 │   │                                                                              │
│   1204 │   │   if addr_table:                                                             │
│   1205 │   │   │   print(                         

This is how the flow we built looks like:
![flow](https://github.com/jina-ai/tutorial-notebooks/blob/main/3D_mesh/flow.png?raw=1)

## Results

Now let's take a look at the search results! Below is the `rifle_16.glb` 3D model we would like to search for:

![Query doc](https://github.com/jina-ai/tutorial-notebooks/blob/main/3D_mesh/query_doc.gif?raw=1)


And the following are the top 3 matches:

![Top 1](https://github.com/jina-ai/tutorial-notebooks/blob/main/3D_mesh/top_1.gif?raw=1)

![Top 2](https://github.com/jina-ai/tutorial-notebooks/blob/main/3D_mesh/top_2.gif?raw=1)

![Top 3](https://github.com/jina-ai/tutorial-notebooks/blob/main/3D_mesh/top_3.gif?raw=1)

**Congratulations!** You have just built a 3D Mesh Search Pipeline!